In [47]:
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score


In [2]:
X_train = pd.read_pickle("data/X_train.pkl")
X_train.head()

,processed_content
53181,moving truck spotted parked outside white hous...
68991,saturday msnbc nbc host chuck todd reacted lew...
60555,tuesday broadcast situation room senator rober...
13536,since sexual assault allegation supreme court ...
66435,jason brian dalton arrested early sunday morni...


In [3]:
X_val = pd.read_pickle("data/X_val.pkl")
X_val.head()

,processed_content
16585,former cricket star turned politician imran kh...
80123,testimony house science committee wednesday mi...
48801,bottom line front judgment former national sec...
9141,fury tropical cyclone pam one powerful storm e...
39116,legislator thing campaigned voter turn back pi...


In [4]:
y_train = pd.read_pickle("data/y_train.pkl")
y_train.head()

53181    0
68991    1
60555    1
13536   -1
66435    1
Name: Bias, dtype: int64

In [6]:
y_val= pd.read_pickle("data/y_val.pkl")
y_val.head()

16585   -1
80123    1
48801    0
9141    -1
39116    0
Name: Bias, dtype: int64

# Vectorize

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [45]:
cv= CountVectorizer(max_features=2000)
cv.fit(X_train["processed_content"])
X_train_cv = cv.transform(X_train["processed_content"])
X_train_cv

<57621x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 9062006 stored elements in Compressed Sparse Row format>

In [46]:
X_train_cv = pd.DataFrame(X_train_cv.toarray(), columns=cv.get_feature_names(), 
                              index=X_train.index)
X_train_cv

MemoryError: Unable to allocate 879. MiB for an array with shape (57621, 2000) and data type int64

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=2000)

tfidf_vectorizer.fit(X_train['processed_content'])

X_train_tf_vec = tfidf_vectorizer.transform(X_train['processed_content'])
X_val_tf_vec = tfidf_vectorizer.transform(X_val['processed_content'])

X_train_tf_vec_df = pd.DataFrame(X_train_tf_vec.toarray(), columns=tfidf_vectorizer.get_feature_names())
X_val_tf_vec_df = pd.DataFrame(X_val_tf_vec.toarray(), columns=tfidf_vectorizer.get_feature_names())

X_train_tf_vec_df.shape

In [ ]:
X_train_tf_vec_df

In [24]:
# Instantiate a MultinomialNB classifier
baseline_model = MultinomialNB()

# Evaluate the classifier on X_train_vectorized and y_train
baseline_cv = cross_val_score(baseline_model, X_train_tf_vec_df, y_train)
baseline_cv

array([0.78229935, 0.76171468, 0.77629295, 0.77785491, 0.78063173])

In [42]:
baseline_model.fit(X_train_tf_vec_df, y_train)
print("Test set Baseline Accuracy: " + str(baseline_model.score(X_train_tf_vec_df, y_train)*100))
print("Validation set Baseline Accuracy: " + str(baseline_model.score(X_val_tf_vec_df, y_val)*100))

Test set Baseline Accuracy: 77.96636642890613
Validation set Baseline Accuracy: 77.41969073775185


In [43]:
baseline_count_cv= cross_val_score(baseline_model, X_train_cv, y_train)
baseline_count_cv

MemoryError: Unable to allocate 879. MiB for an array with shape (46096, 2500) and data type int64

In [44]:
baseline_model.fit(X_train_cv, y_train)
print("Test set Baseline Accuracy: " + str(baseline_model.score(X_train_cv, y_train)*100))
print("Validation set Baseline Accuracy: " + str(baseline_model.score(X_val_cv, y_val)*100))

MemoryError: Unable to allocate 1.07 GiB for an array with shape (57621, 2500) and data type float64

## Modeless Predictions

In [23]:
y_train.value_counts(normalize= True)

 1    0.348189
-1    0.328769
 0    0.323042
Name: Bias, dtype: float64